In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib

In [ ]:
df1 = pd.read_csv("../input/bengaluru-house-price-data/Bengaluru_House_Data.csv")
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.groupby("area_type")["area_type"].agg("count")

In [ ]:
df2=df1.drop(["availability","society","balcony"],axis="columns")

In [ ]:
df2.head()

In [ ]:
df2.isnull().sum()

In [ ]:
df2['bath']=df2['bath'].fillna(df2['bath'].mean())
df2.isnull().sum()

In [ ]:
df3=df2.dropna()
df3.isnull().sum()

In [ ]:
df3['size'].unique()

In [ ]:
df3['bhk']=df3['size'].apply(lambda x: int(x.split(" ")[0]))
df3.head()

In [ ]:
df3['bhk'].unique()

In [ ]:
df3['total_sqft'].unique()

In [ ]:
def convert_sqft_to_no(x):
    try:
        if(len(x.split("-"))==2):
           return float(x[0]+x[1]/2)
        else:
            return float(x)
    
    except:
       return None

In [ ]:
df4=df3.copy()
df4['total_sqft']=df4['total_sqft'].apply(convert_sqft_to_no)
df4.head()

In [ ]:
df5=df4.copy()

In [ ]:
df5['price_per_sqft']=(df5['price']*100000)/df5['total_sqft']

In [ ]:
df5.head()

In [ ]:
df5["location"]=df5.location.apply(lambda x: x.strip())

In [ ]:
location_stats=df5.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

In [ ]:
len(location_stats[location_stats<=5])

In [ ]:
location_stats_less_than_10=location_stats[location_stats<=10]
location_stats_less_than_10

In [ ]:
df5['location']=df5.location.apply(lambda x:'other'if x in location_stats_less_than_10 else x)
len(df5['location'].unique())

In [ ]:
df5.head()


In [ ]:
df5[df5["total_sqft"]/df5["bhk"]<300]

In [ ]:
df6=df5[~(df5["total_sqft"]/df5["bhk"]<300)]
df6.shape

In [ ]:
df6['price_per_sqft'].describe()

In [ ]:
def remove_pps_outliers(df):
    res_df=pd.DataFrame()
    for i,sub_df in df.groupby('location'):
        m=np.mean(sub_df.price_per_sqft)
        sd=np.std(sub_df.price_per_sqft)
        reduced_df=sub_df[(sub_df.price_per_sqft>(m-sd)) & (sub_df.price_per_sqft<=(m+sd))]
        res_df=pd.concat([res_df,reduced_df],ignore_index=True)
    return res_df

df7=remove_pps_outliers(df6)
df7.head()

In [ ]:
df7.shape

In [ ]:
def plot_scatter_chart(df,location):
    bhk_2=df[(df.location==location) & (df.bhk==2)]
    bhk_3=df[(df.location==location) & (df.bhk==3)]
    matplotlib.rcParams['figure.figsize']=(15,10)
    plt.scatter(bhk_2.total_sqft,bhk_2.price,color='blue',label='2 BHK',s=50)
    plt.scatter(bhk_3.total_sqft,bhk_3.price,marker='+',color='green',label='3 BHK',s=50)
    plt.xlabel("Total sqft Area")
    plt.ylabel("price")
    plt.title(location)
    plt.legend()
    

In [ ]:
plot_scatter_chart(df7,"Rajaji Nagar")

In [ ]:
plot_scatter_chart(df7,"Whitefield")

<h3>we can remove <i>Greater</i> BHK appartments whose <u>price per square foot</u> is <b>less</b> than that of <i>Lesser</i> BHK appartments.

In [ ]:
def remove_bhk_outliers(df):
    exclude_indices=np.array([])
    for location,location_df in df.groupby('location'):
        bhk_stats={}
        for bhk,bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk]={
                'mean':np.mean(bhk_df.price_per_sqft),
                'std':np.std(bhk_df.price_per_sqft),
                'count':bhk_df.shape[0]
            }
        for bhk,bhk_df in location_df.groupby('bhk'):
            stats=bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices=np.append(exclude_indices,bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df8=remove_bhk_outliers(df7)
df8.shape

In [ ]:
plot_scatter_chart(df8,'Rajaji Nagar')

In [ ]:
plot_scatter_chart(df8,'Whitefield')

In [ ]:
plot_scatter_chart(df8,'Hebbal')

In [ ]:
plot_scatter_chart(df7,"Hebbal")

In [ ]:
matplotlib.rcParams["figure.figsize"] = (20,10)
plt.hist(df8.price_per_sqft,rwidth=.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

In [ ]:
df8.bath.unique()

In [ ]:
df8[df8.bath>10]

In [ ]:
plt.hist(df8.bath,rwidth=0.8)
plt.xlabel("Number of Bathrooms")
plt.ylabel("Count")

In [ ]:
df8[df8.bath>df8.bhk+2]

In [ ]:
df9=df8[df8.bath<df8.bhk+2]
df9.shape

In [ ]:
df10=df9.drop(['size','price_per_sqft'],axis='columns')

In [ ]:
df10.head()

In [ ]:
dummies=pd.get_dummies(df10.location)
dummies.head()

In [ ]:
df12=pd.concat([df10,dummies.drop('other',axis='columns')],axis='columns')
df12.head()

In [ ]:
df13=df12.drop(['area_type','location'],axis='columns')

In [ ]:
df13.shape

In [ ]:
X=df13.drop('price',axis='columns')
X.head()

In [ ]:
y=df13['price']
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)


In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)
lr_score=lr.score(x_test,y_test)

# Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso
lr_lasso=Lasso()
lr_lasso.fit(x_train,y_train)
lr_lasso_score=lr_lasso.score(x_test,y_test)


# SUPPORT VECTOR MACHINE

In [ ]:
from sklearn.svm import SVR
svr = SVR()
svr.fit(x_train,y_train)
svr_score=svr.score(x_test,y_test)

# RANDOM FOREST REGRESSOR

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(x_train,y_train)
rfr_score=rfr.score(x_test,y_test)

In [ ]:
print(pd.DataFrame([{'Model': 'Linear Regression','Score':lr_score},
              {'Model': 'Lasso','Score':lr_lasso_score},
              {'Model': 'Support Vector Machine','Score':svr_score},
              {'Model': 'Random Forest','Score':rfr_score}],
             columns=['Model','Score']))

In [ ]:
def predict_price(location,sqft,bath,bhk):
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return lr.predict([x])[0]

In [ ]:
predict_price('Vijayanagar',1000,2,2)

In [ ]:
X.columns

In [ ]:
predict_price('1st Phase JP Nagar',1450, 4, 4)

In [ ]:
predict_price('1st Phase JP Nagar',1000, 1, 2)

In [ ]:
predict_price('Whitefield',1200, 2, 2)

In [ ]:
predict_price('Whitefield',2000, 2, 2)

In [ ]:
predict_price('Rajaji Nagar',1640, 3, 3)

In [ ]:
y_pred = lr.predict(x_test)

In [ ]:
coeff_df = pd.DataFrame(lr.coef_, df13.drop('price',axis =1).columns, columns=['Coefficient']) 
coeff_df

In [ ]:
import seaborn as sns
fig = plt.figure(figsize=(10,5))
residuals = (y_test- y_pred)
sns.distplot(residuals)

In [ ]:

# evaluate the performance of the algorithm (MAE - MSE - RMSE)
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))  
print('MSE:', metrics.mean_squared_error(y_test, y_pred))  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('VarScore:',metrics.explained_variance_score(y_test,y_pred))

In [ ]:
#compare actual output values with predicted values
y_pred = lr.predict(x_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df1 = df.head(10)
df1

# Keras Regression

In [ ]:
#pip install tensorflow

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
# having 4 neuron is based on the number of available features
model = Sequential()
model.add(Dense(245,activation='relu'))
model.add(Dense(500,activation='relu'))
model.add(Dense(500,activation='relu'))
model.add(Dense(500,activation='relu'))
model.add(Dense(1))
model.compile(optimizer='Adam',loss='mse')

In [ ]:
model.fit(x=x_train,y=y_train,validation_data=(x_test,y_test),batch_size=256,epochs=500)
model.summary()

In [ ]:
loss_df = pd.DataFrame(model.history.history)
loss_df.plot(figsize=(12,8))

In [ ]:
y_pred = model.predict(x_test)
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))  
print('MSE:', metrics.mean_squared_error(y_test, y_pred))  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

print('VarScore:',metrics.explained_variance_score(y_test,y_pred))

In [ ]:
# Visualizing Our predictions
fig = plt.figure(figsize=(10,5))
plt.scatter(y_test,y_pred)
# Perfect predictions
plt.plot(y_test,y_test,'r')

In [ ]:
import pickle
with open('REPP.pickle','wb') as f:
    pickle.dump(lr,f)

In [ ]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))